In [54]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

## Data

In [55]:
#Import the data
fashion_mnist_data, fashion_mnist_info = tfds.load(name='fashion_mnist',with_info=True,as_supervised=True)


#Split into train,test variables
fashion_train, fashion_test = fashion_mnist_data['train'],fashion_mnist_data['test']

#Calculate validation samples count (10% of train data)
num_validation_samples = len(fashion_train) * .1

#Cast the value to a proper Int 
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

In [56]:
#Function to scale the data.
#To be passed as arguement to map.
def Scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255
    return image,label


In [154]:
#Scale our data: (We get a value between 0 and 1)

scaled_train_and_validation_data = fashion_train.map(Scale)
scaled_test_data = fashion_test.map(Scale)

#Shuffle the data:

BUFFER_SIZE = 10000
fashion_train_and_validation_shuffled = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


#Split train/ validation:
validation_data = fashion_train_and_validation_shuffled.take(num_validation_samples)
train_data = fashion_train_and_validation_shuffled.skip(num_validation_samples)


#Batch train data:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

#We don't require to batch the validation and test, but the model expects it
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(len(scaled_test_data))

validation_inputs, validation_outputs = next(iter(validation_data))

## Model

### Outline model

In [155]:
output_size = 10
hidden_layer_width = 50

#Create the model:
model = tf.keras.Sequential(
                [
                    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(28,28,1)),
                    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(hidden_layer_width,activation='relu'),
                    tf.keras.layers.Dense(output_size,activation='softmax')
                    
                ])

### Optimizer and loss

In [156]:
model.compile(optimizer=tf.optimizers.Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Training

In [157]:
NUM_EPOCHS = 10

model.fit(train_data,epochs=NUM_EPOCHS,verbose=2,validation_data=(validation_inputs,validation_outputs))


Epoch 1/10
540/540 - 16s - loss: 0.4827 - accuracy: 0.8357 - val_loss: 0.3506 - val_accuracy: 0.8777
Epoch 2/10
540/540 - 14s - loss: 0.3183 - accuracy: 0.8882 - val_loss: 0.3060 - val_accuracy: 0.8905
Epoch 3/10
540/540 - 15s - loss: 0.2788 - accuracy: 0.9020 - val_loss: 0.2722 - val_accuracy: 0.8977
Epoch 4/10
540/540 - 16s - loss: 0.2535 - accuracy: 0.9099 - val_loss: 0.2640 - val_accuracy: 0.9017
Epoch 5/10
540/540 - 16s - loss: 0.2352 - accuracy: 0.9162 - val_loss: 0.2472 - val_accuracy: 0.9113
Epoch 6/10
540/540 - 16s - loss: 0.2177 - accuracy: 0.9215 - val_loss: 0.2423 - val_accuracy: 0.9102
Epoch 7/10
540/540 - 16s - loss: 0.2035 - accuracy: 0.9273 - val_loss: 0.2109 - val_accuracy: 0.9207
Epoch 8/10
540/540 - 17s - loss: 0.1881 - accuracy: 0.9320 - val_loss: 0.2192 - val_accuracy: 0.9153
Epoch 9/10
540/540 - 16s - loss: 0.1775 - accuracy: 0.9359 - val_loss: 0.1861 - val_accuracy: 0.9297
Epoch 10/10
540/540 - 15s - loss: 0.1655 - accuracy: 0.9407 - val_loss: 0.1797 - val_accura

In [158]:
#Test accuracy is 91%, this could mean we are slighly overfitting on the train data
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.2587 - accuracy: 0.9130
